In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
from random import random as rd 
import matplotlib.pyplot as plt
from mesa.datacollection import DataCollector
import numpy as np
import math
from scipy.special import erf #especial para la generación del maxboltzdist
from scipy.interpolate import interp1d as interp #especial para la generación del maxboltzdist
import time

j=0
for j in range(40): #qui se coloca el número de veces que quiere que se cree una muestra
    class SALÓN(Model):
        #Inicia el modelo con el número esepcificado de agentes, cantidad de celdas.
        def __init__(self, N):
            self.num_agents = N
            self.schedule = RandomActivation(self)
            # Crea a los actores.
            for i in range(self.num_agents):
                a = Actor(i,self)
                self.schedule.add(a)
                self.datacollector = DataCollector(
                agent_reporters={"Lista": lambda a: a.lista,
                             "personalidad": lambda a: a.personalidad,
                             "conocidos": lambda a: a.conocidos, "amistad": lambda a: a.amistad,
                             "proba_de_cono": lambda a: a.probai,"enlace_pref": lambda a: a.pij})
            
        # se activa todo lo que se definió en la parte de la clase "Salon"
        def step(self):
            self.datacollector.collect(self)
            self.schedule.step()
        
    class Actor(Agent):
        """ An agent with fixed initial wealth."""
        def __init__(self, unique_id, model):
            super().__init__(unique_id, model)
            ##############################################################
            amu = 1.66e-27
            mass = 1.0079*amu
            T =100
            def MB_CDF(v,m,T):
                #Función cumulativa de distribución de la distribución de Maxwell-Boltzmann"""
                kB = 1.38e-23
                a = np.sqrt(kB*T/m)
                return erf(v/(np.sqrt(2)*a)) - np.sqrt(2/np.pi)* v* np.exp(-v**2/(2*a**2))/a

            # se crea CDF
            vs = np.arange(0,11000)#hasta que número va a permitir que se genere en x
            cdf = MB_CDF(vs,mass,T) # esencialmente y = f(x)

            #función de interpolación de la CDF
            inv_cdf = interp(cdf,vs) # lo que obtenemos es un  x = g(y) from y = f(x)

            def generate_velocities(n):
                rand_nums = np.random.random(n)
                speeds = inv_cdf(rand_nums)
                return speeds

            def MB_speed(v,m,T):
                #Función de distribución de Maxwell-Boltzmann
                kB = 1.38e-23
                return (m/(2*np.pi*kB*T))**1.5 * 4*np.pi * v**2 * np.exp(-m*v**2/(2*kB*T))

            spd= generate_velocities(1) #genera los números de la distribución en x
            ##############################################
            self.personalidad = spd[0]/10000 #para normalizar entre 0 y 1
            self.iniciador = 0
            self.lista = []
            self.conocidos = {}
            self.eta = 1 
            self.amistad = {}
            self.proba1 = rd() #número aleatorio entre 0 y 1 con dist. unif. para pi
            self.proba2 = rd() #número aleatoriio entre 0 y 1 con dist. unif. para pij
            self.probai = 0
            self.pij = 0 
            
        # se define bajo que condiciones se agregarán a sus listas
        def agregar_a_la_lista(self,b):
            self.lista.append(b.unique_id)
            b.lista.append(self.unique_id)
                
        #se define como es que se sabrán la cantidad de contactos que tuvieron con un determinado agente
        def cantidad_de_contactos(self):#para crear el diccionario de número de contactos 
            a = len(self.lista) 
            i = 0
            while i <= a-1:
                b=self.lista[i]
                c=self.lista.count(b)
                self.conocidos.update({b:c}) # actualiza el key en caso de existir o lo crea
                i+=1
            #print(self.conocidos)
        
        def amistad_fij(self): # para crear la amistad fij que se tiene con cada actor j
            a = len(self.lista) 
            i = 0
            while i <= a-1:
                b = self.lista[i]
                c = self.conocidos.get(b)
                fij = str(1- math.exp(-c*self.eta))
                self.amistad.update({b:fij[:fij.find(".")+5]}) # actualiza el key en caso de existir o lo crea
                i+=1
        
        def proba_para_la_lista(self): # proba para contactar alguien dentro de la lista 
            item = 0
            ################## pequeñito progama para obtener ki
            for k,v in self.amistad.items(): 
                item = item+1
            #################
            a = item
            b = str(1- math.exp(-(1/self.personalidad)*a*0.05))####### aquí se agrega el factor 1/4 para que tenga
                                                         ####### que los de mayor velocidad conozcan más personas
            self.probai = float(b[:b.find(".")+5])
                
        def enlace_preferencial(self,b):
            f_ij = self.amistad
            ################## pequeñito programa de la sumatoria que da como resultado a f_ik
            f_ik = 0
            for k,v in f_ij.items(): 
                dato = f_ij.get(k)
                f_ik = f_ik + float(dato)
            ##################
            d = str(float(f_ij.get(b))/f_ik) # no es un diccionario
            self.pij = float(d[:d.find(".")+5])
        
        def step(self):# este contiene todas las funciones a hacer antes de moverlos de lugar
        
            contenido = self.model.schedule.agents    
            self.proba_para_la_lista() # el 1 
            otro = random.choice(contenido)
            while self.unique_id == otro.unique_id:
                otro = random.choice(contenido)
            if float(self.probai) >= self.proba1 and otro.unique_id in self.lista:
                self.enlace_preferencial(otro.unique_id) # el 2
                if float(self.pij) >= self.proba2: 
                    self.agregar_a_la_lista(otro)# el 3.1
                    self.cantidad_de_contactos() # el 4
                    self.amistad_fij() # el 5
                    otro.cantidad_de_contactos() # el 4
                    otro.amistad_fij() # el 5
            elif float(self.probai) < self.proba1 and not otro.unique_id in self.lista:
                self.agregar_a_la_lista(otro)# el 3.1
                self.cantidad_de_contactos() # el 4
                self.amistad_fij() # el 5
                otro.cantidad_de_contactos() # el 4
                otro.amistad_fij() # el 5
            
            self.iniciador = self.iniciador+1 # en el primer paso los actores pueden tener contacto entre 3, pero 
            #para el segundo ya no debido a esta variable, teniendo control de que sean solo 2 
        
            #ajustar el self.iniciador de acuerdo a la cnatidad de veces que se iterará el programa
            if self.iniciador == 200: ##### si no se cambia este número al número de veces que correra la sumulación
                ##### los archivos a crear siguientes se aparecerán vacios
                h = open("dato_tesis" +str(j) +".csv", "a") # borrar archivo cada vez que se quiera crear uno nuevo
                for k,v in self.conocidos.items():
                    h.write("{} {} {} {}\n".format(self.unique_id,k,self.personalidad,v))
                h.close()
                f = open("enlaces" +str(j) +".txt", "a") # borrar archivo cada vez que se quiera crear uno nuevo
                for k,v in self.conocidos.items():
                    f.write("{} {}\n".format(self.unique_id,k))
                f.close()
            
    # run.py
    h = open("dato_tesis" +str(j) +".csv", "a")
    h.write("{} {} {} {}\n".format("P(k)","k","Personalidad","Cantidad"))
    h.close()

    a = 10000 #numero de actores
    model = SALÓN(a) #El primer espacio es la cantidad de actores, el segundo las dimensones de 'x' y el tercero 
                    #las de 'y'
    print(time.ctime(time.time()))
    print (j)
    for h in range(200): #número de veces que correa una simulación
        model.step()
    print(time.ctime(time.time()))

In [ ]:
resumen = open("resumen.csv", "a")
resumen.write("{} {} {} {} {} {} {} \n".format("m","b","R^2","Entropia","Kmax","Kmin","Personalidad"))
resumen.close()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats 
######################################################################
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
plt.rc('figure', titlesize=18)  # fontsize of the figure title
####################################################################
i=0
for i in range (40):

    datos= pd.read_csv("dato_tesis" +str(i) +".csv", sep=' ')

########################### primer pivote para las varias varibles con respecto a P(k)
    c=datos.pivot_table(values=['k'],index=['P(k)'], aggfunc=[np.size]) 
    d=datos.pivot_table(values=['Personalidad'],index=['P(k)'], aggfunc=[np.max]) #forma 2
    e=datos.pivot_table(values=['Cantidad'],index=['P(k)'], aggfunc=[np.size]) #forma 3
    tabla1=pd.concat((c,d,e), axis=1) #unir todas las tablas

    tabla1.columns = tabla1.columns.droplevel(0)
    tabla1 = tabla1.reset_index().rename_axis(None, axis=1)
    orden=tabla1.sort_values(by=['Personalidad'])
    orden=orden.head(20)
    orden.to_csv(r"Extrovertidos" +str(i) +".csv")############## guardar tabla como csv

    imagen1=tabla1.plot(kind='scatter', x='k', y='Personalidad',
                title='Distribución de Personalidad vs. Grado',
               figsize=(10, 5))# para ver distribución de la personalidad con respecto al grado 
    plt.savefig("Dist Pers vs. K" +str(i) +".png")

########################### segundo pivote
    f=tabla1.pivot_table(values=['P(k)'],index=['k'], aggfunc=np.size)
    g=tabla1.pivot_table(values=['Personalidad'],index=['k'], aggfunc=np.mean)
    tabla2=pd.concat((f,g), axis=1) #unir todas las tablas
    tabla2 = tabla2.reset_index().rename_axis(None, axis=1)
    tabla2['P(k)']=tabla2['P(k)']/10000 
    tabla2.to_csv(r"Distribuciónes" +str(i) +".csv")############## guardar tabla como csv
    entropy = stats.entropy(tabla2['P(k)'])
    máximo=np.max(tabla2['k'])
    su_persona=tabla2["Personalidad"].iloc[-1]
    mínimo=np.min(tabla2['k'])

#############################################################
    imagen2=tabla2.plot(kind='scatter', x='k', y='P(k)', loglog=True, c='Personalidad',
                       colormap='viridis',title='Distribución de Grado',figsize=(10, 5))
    plt.xlabel("Log (k)")
    imagen2.set_ylabel("Log (P(k))")
    plt.savefig("Distribución_log" +str(i) +".png")

#####################################################################
    imagen3=tabla2.plot.scatter(x='k', y='P(k)', c='Personalidad',
                       colormap='viridis',title='Distribución de Grado',figsize=(10, 5))
    imagen3.set_xlabel("k")
    imagen3.set_ylabel("P(k)")
    plt.savefig("Distribución" +str(i) +".png")
#########################################################
    tabla2['P(k)']=np.log(tabla2['P(k)']) 
    tabla2['k']=np.log(tabla2['k'])
    X = tabla2['k']
    y = tabla2['P(k)']

    X = tabla2.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
    Y = tabla2.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions

##############################################################
    imagen4=tabla2.plot(kind='scatter', x='k', y='P(k)',
                       title='Distribución de Grado',figsize=(10,5))
    plt.scatter(X, Y)
    plt.plot(X, Y_pred, '-r',label="Ajuste")
    plt.legend(loc='upper right')
    imagen4.set_xlabel("Log(k)")
    imagen4.set_ylabel("Log(P(k))")
    plt.savefig("Distribución_log_ajuste" +str(i) +".png")

################### funciona con scipy
    slope, intercept, r_value, p_value, std_err = stats.linregress(tabla2['k'], tabla2['P(k)'])
###################
    resumen = open("resumen.csv", "a")
    resumen.write("{} {} {} {} {} {} {}\n".format(slope,intercept,r_value**2,entropy,
                                    máximo,mínimo,su_persona))
    resumen.close()



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

######################################################################
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
plt.rc('figure', titlesize=18)  # fontsize of the figure title
####################################################################

i=0
for i in range (40):

    datos= pd.read_csv("dato_tesis" +str(i) +".csv", sep=' ')

########################### primer pivote para las varias varibles con respecto a P(k)
    tabla1=datos.pivot_table(values=['k'],index=['P(k)'], aggfunc=[np.size]) 
    tabla1.columns = tabla1.columns.droplevel(0)
    tabla1 = tabla1.reset_index().rename_axis(None, axis=1)

########################### segundo pivote
    tabla2=tabla1.pivot_table(values=['P(k)'],index=['k'], aggfunc=np.size)
    tabla2 = tabla2.reset_index().rename_axis(None, axis=1)
    tabla2['P(k)']=tabla2['P(k)']/10000 
    
#####################################################################
    imagen3=tabla2.plot.scatter(x='k', y='P(k)',
                       title='Distribución de Grado',figsize=(10, 5))
    imagen3.set_xlabel("k")
    imagen3.set_ylabel("P(k)")
    plt.savefig("Distribución_sin_personalidad" +str(i) +".png")
    #plt.show()
########################################################
   


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats 

######################################################################
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=18)    # legend fontsize
plt.rc('figure', titlesize=18)  # fontsize of the figure title
####################################################################

for i in range (40):

    datos= pd.read_csv("Distribuciónes"+str(i)+".csv", sep=',')
    #datos.drop(datos.columns['Personalidad'], axis=1)
    a= datos['P(k)']
    b= datos['k']
    datos=pd.concat((a,b), axis=1) #unir todas las tablas
###################################   
    x = datos['P(k)']

    plt.close('all') # antes de graficar, cierro todos las figuras que estén abiertas
    bins=1000 # número de bins del histograma
    plt.figure(figsize=(10,5))
    n,bin_positions,p = plt.hist(x,bins,density=True, stacked=True) # grafico el histograma

    #le pongo nombres a los ejes
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Log(k)')
    plt.ylabel('Log(P(k))')
    plt.title('Histograma de Distribución de Grado')
    plt.savefig("histograma" +str(i) +".png")
    print(i)
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats 

resumen = open("ni_por_Ei.csv", "a")
resumen.write("{} {}\n".format("Muestra","U[J]"))
resumen.close()


for i in range (40):

    datos= pd.read_csv("dato_tesis" +str(i) +".csv", sep=' ')
    d=datos.pivot_table(values=['Personalidad'],index=['P(k)'], aggfunc=[np.max]) #Tabla dinámica
    d.columns = d.columns.droplevel(0)#####reajustar a tablar normal llamada d
    d=d.reset_index().rename_axis(None, axis=1)#####siguiendo con el reajuste de tabla llamado d
    amu = 1.66e-27
    mass = 1.0079*amu
    x = (d['Personalidad']**2)*mass*0.5*(10000*10000)
    #print(i,np.sum(x))
    resumen = open("ni_por_Ei.csv", "a")
    resumen.write("{} {}\n".format(i,np.sum(x)))
    resumen.close()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
##### solo sirve para escribir el promedio y desviación estandar de la energía en las muestras usadas
datos= pd.read_csv("ni_por_Ei.csv", sep=' ')
#print(datos)
#datos.drop(datos.columns['Personalidad'], axis=1)
x= datos['U[J]']
print(np.mean(x),np.std(x))
    